In [1]:
import numpy as np
from math import ceil, sqrt, pi, atan2
import sys
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import ipywidgets as ipw
from IPython.display import display

%matplotlib widget

In [2]:
def Coulomb(wn, d, x0, v0):
    def displ(x0, v0, t):
        if v0 > 0.0:
            return (x0-d)*np.cos(wn*t) + v0/wn*np.sin(wn*t) + d
        elif v0 < 0.0:
            return (x0+d)*np.cos(wn*t) + v0/wn*np.sin(wn*t) - d
        elif x0 > 0.0:
            return (x0-d)*np.cos(wn*t) + d
        else:
            return (x0+d)*np.cos(wn*t) - d

    taun = 2*pi/wn
    tau2 = taun/2
    spc = 81
    times = []
    x = []
    xlast = x0
    ticks = []
    
    if v0 != 0.0:
        tf = atan2(v0, wn*(x0-d)) if v0 > 0.0 else atan2(v0, wn*(x0+d))
        if tf <0: tf += pi
        tf /= wn
        nsamp = int(tf/tau2*spc)
        ti = np.linspace(0, tf, nsamp)
        xi = displ(x0, v0, ti)
        times.append(ti)
        x.append(xi)
        xlast = xi[-1]
        ticks.append(tf)

    while abs(xlast) > d:
        ti = np.linspace(0, tau2, spc)
        xi = displ(xlast, 0, ti)
        ti += times[-1][-1]
        times.append(ti[1:])
        x.append(xi[1:])
        xlast = xi[-1]
        ticks.append(ti[-1])
        
    #return np.hstack(times), np.stack(x)
    return np.hstack(times), np.hstack(x), ticks
    

In [6]:
style = {'description_width': 'initial'}
layout = ipw.Layout(width="100px")
mw = ipw.BoundedFloatText(
    value=1.0,
    min=0,
    max=sys.float_info.max,
    step=0.5,
    disabled=False,
    layout=layout
)
cw = ipw.BoundedFloatText(
    value=1.2,
    min=0,
    max=sys.float_info.max,
    step=0.05,
    disabled=False,
    layout=layout
)
kw = ipw.BoundedFloatText(
    value=40,
    min=0,
    max=sys.float_info.max,
    step=0.5,
    disabled=False,
    layout=layout
)

line1 = ipw.HBox([
    ipw.HBox([ipw.Label(value="Massa:", style=style), mw]),
    ipw.HBox([ipw.Label(value="Amortecimento:", style=style), cw]),
    ipw.HBox([ipw.Label(value="Rigidez:", style=style), kw])
    ])

Nw = ipw.BoundedFloatText(
    value=25,
    step=0.1,
    disabled=False,
    layout=layout
)
muw = ipw.BoundedFloatText(
    value=0.125,
    step=0.005,
    min=0.0,
    max=1.0,
    disabled=False,
    layout=layout
)

line2 = ipw.HBox([
    ipw.HBox([ipw.Label(value=r"Coef. de atrito ($\mu$):", style=style), muw]),
    ipw.HBox([ipw.Label(value=r"Força Normal", style=style), Nw])
    ])
x0w = ipw.BoundedFloatText(
    value=5,
    step=0.1,
    disabled=False,
    layout=layout
)
v0w = ipw.BoundedFloatText(
    value=25,
    step=0.1,
    disabled=False,
    layout=layout
)

line3 = ipw.HBox([
    ipw.HBox([ipw.Label(value=r"Posição inicial:", style=style), x0w]),
    ipw.HBox([ipw.Label(value=r"Velocidade inicial:", style=style), v0w])
    ])
accordion = ipw.Accordion(children=[line1, line2, line3])
accordion.set_title(0, 'Sistema Mecânico')
accordion.set_title(1, 'Atrito')
accordion.set_title(2, 'Condições Iniciais')

In [8]:
def make_plot(m, c, k, x0, v0, mu, N):
    d = mu*N/k
    wn = sqrt(k/m)
    tau2 = pi/wn
    times, x, ticks = Coulomb(wn=wn, d=d, x0=x0, v0=v0)

    fig, ax = plt.subplots()
    fig.set_size_inches(10, 6) 
    plt.title("Atrito de Coulomb")
    plt.xlabel("Tempo (s)")
    plt.ylabel("Deslocamento (m)")
    plt.xticks(ticks)
    ax.plot(times, x)
    ax.grid(True)

    # Set major x-axis ticks every 2 units
    plt.axhline(y=d, color='r', linestyle='--', label=r'$\frac{\mu N}{k}$')
    plt.axhline(y=-d, color='r', linestyle='--', label=r'$-\frac{\mu N}{k}$')
    plt.xlim(0, None)
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()


interactive_plot = ipw.interactive_output(make_plot,
    {'m':mw, 'c':cw, 'k':kw, 'x0':x0w, 'v0':v0w, 'mu':muw, 'N':Nw})
display(accordion, interactive_plot)

Accordion(children=(HBox(children=(HBox(children=(Label(value='Massa:', style=LabelStyle(description_width='in…

Output()